# Project Overview

Project Overview
The objective of this challenge is to predict the pure fire premium for agricultural insurance policies by modelling:

- Claim frequency

- Average claim cost

The final target, the claim burden, is computed as:
'frequency × average cost × ANNEE_ASSURANCE' (years since policy subscription).

# 0 - Import Packages    

In [33]:
# Importer les bibliothèques nécessaires
import pandas as pd
pd.set_option('display.max_columns', 375)
import numpy as np
from category_encoders import CountEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.svm import SVR

# 1- Open Files

In [35]:
# Charger les données
print("Chargement des données...")
X_train = pd.read_csv(r"C:\Users\HP\Downloads\DescriptifNotebookExemple\train_input_Z61KlZo.csv")
y_train = pd.read_csv(r"C:\Users\HP\Downloads\DescriptifNotebookExemple\train_output_DzPxaPY.csv")
X_test = pd.read_csv(r"C:\Users\HP\Downloads\DescriptifNotebookExemple\test_input_5qJzHrr.csv")
print("Données chargées avec succès.")

Chargement des données...


C:\Users\HP\AppData\Local\Temp\ipykernel_22136\3404366266.py:3: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv(r"C:\Users\HP\Downloads\DescriptifNotebookExemple\train_input_Z61KlZo.csv")
C:\Users\HP\AppData\Local\Temp\ipykernel_22136\3404366266.py:5: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv(r"C:\Users\HP\Downloads\DescriptifNotebookExemple\test_input_5qJzHrr.csv")


Données chargées avec succès.


In [36]:
# Afficher un aperçu des données pour vérifier leur chargement
print("Aperçu de X_train :")
display(X_train.head())
print("Aperçu de y_train :")
display(y_train.head())
print("Aperçu de X_test :")
display(X_test.head())

Aperçu de X_train :


,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,CARACT5,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG8,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,CA1,CA2,CA3,KAPITAL1,KAPITAL2,KAPITAL3,KAPITAL4,KAPITAL5,KAPITAL6,KAPITAL7,KAPITAL8,KAPITAL9,KAPITAL10,KAPITAL11,KAPITAL12,KAPITAL13,KAPITAL14,KAPITAL15,KAPITAL16,KAPITAL17,KAPITAL18,KAPITAL19,KAPITAL20,KAPITAL21,KAPITAL22,KAPITAL23,KAPITAL24,KAPITAL25,KAPITAL26,KAPITAL27,KAPITAL28,KAPITAL29,KAPITAL30,KAPITAL31,KAPITAL32,KAPITAL33,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE2,SURFACE3,SURFACE4,SURFACE5,SURFACE6,SURFACE7,SURFACE8,SURFACE9,SURFACE10,SURFACE11,SURFACE12,SURFACE13,SURFACE14,SURFACE15,SURFACE16,SURFACE17,SURFACE18,SURFACE19,SURFACE20,SURFACE21,NBBAT1,NBBAT2,NBBAT3,NBBAT4,NBBAT5,NBBAT6,NBBAT7,NBBAT8,NBBAT9,NBBAT10,NBBAT11,NBBAT13,NBBAT14,TAILLE3,TAILLE4,NBSINCONJ,NBSINSTRT,COEFASS,RISK1,RISK2,RISK3,RISK4,RISK5,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT1,EQUIPEMENT2,EQUIPEMENT3,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,EQUIPEMENT7,DISTANCE_111,DISTANCE_112,DISTANCE_121,DISTANCE_122,DISTANCE_123,DISTANCE_124,DISTANCE_131,DISTANCE_132,DISTANCE_133,DISTANCE_141,DISTANCE_142,DISTANCE_211,DISTANCE_212,DISTANCE_213,DISTANCE_221,DISTANCE_222,DISTANCE_223,DISTANCE_231,DISTANCE_242,DISTANCE_243,DISTANCE_244,DISTANCE_311,DISTANCE_312,DISTANCE_313,DISTANCE_321,DISTANCE_322,DISTANCE_323,DISTANCE_324,DISTANCE_331,DISTANCE_332,DISTANCE_333,DISTANCE_334,DISTANCE_335,DISTANCE_411,DISTANCE_412,DISTANCE_421,DISTANCE_422,DISTANCE_423,DISTANCE_511,DISTANCE_512,DISTANCE_521,DISTANCE_522,DISTANCE_523,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_14,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_INC,LOG_SOC,IND,IND_0_Y1,IND_Y1_Y2,IND_Y2_Y3,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y6_Y7,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_INC,IND_SNV,MEN_SURF,DISTANCE_1,DISTANCE_2,ALTITUDE_1,ALTITUDE_2,ALTITUDE_3,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,HAUTEUR,HAUTEUR_MAX,BDTOPO_BAT_MAX_HAUTEUR,ZONE_VENT,NB_CASERNES,NBJTX25_MM_A,NBJTX25_MMAX_A,NBJTX25_MSOM_A,NBJTX0_MM_A,NBJTX0_MMAX_A,NBJTX0_MSOM_A,NBJTXI27_MM_A,NBJTXI27_MMAX_A,NBJTXI27_MSOM_A,NBJTXS32_MM_A,NBJTXS32_MMAX_A,NBJTXS32_MSOM_A,NBJTXI20_MM_A,NBJTXI20_MMAX_A,NBJTXI20_MSOM_A,NBJTX30_MM_A,NBJTX30_MMAX_A,NBJTX30_MSOM_A,NBJTX35_MM_A,NBJTX35_MMAX_A,NBJTX35_MSOM_A,NBJTN10_MM_A,NBJTN10_MMAX_A,NBJTN10_MSOM_A,NBJTNI10_MM_A,NBJTNI10_MMAX_A,NBJTNI10_MSOM_A,NBJTN5_MM_A,NBJTN5_MMAX_A,NBJTN5_MSOM_A,NBJTNS25_MM_A,NBJTNS25_MMAX_A,NBJTNS25_MSOM_A,NBJTNI15_MM_A,NBJTNI15_MMAX_A,NBJTNI15_MSOM_A,NBJTNI20_MM_A,NBJTNI20_MMAX_A,NBJTNI20_MSOM_A,NBJTNS20_MM_A,NBJTNS20_MMAX_A,NBJTNS20_MSOM_A,NBJTMS24_MM_A,NBJTMS24_MMAX_A,NBJTMS24_MSOM_A,TAMPLIAB_VOR_MM_A,TAMPLIAB_VOR_MMAX_A,TAMPLIM_VOR_MM_A,TAMPLIM_VOR_MMAX_A,TM_VOR_MM_A,TM_VOR_MMAX_A,TMM_VOR_MM_A,TMM_VOR_MMAX_A,TMMAX_VOR_MM_A,TMMAX_VOR_MMAX_A,TMMIN_VOR_MM_A,TMMIN_VOR_MMAX_A,TN_VOR_MM_A,TN_VOR_MMAX_A,TNAB_VOR_MM_A,TNAB_VOR_MMAX_A,TNMAX_VOR_MM_A,TNMAX_VOR_MMAX_A,TX_VOR_MM_A,TX_VOR_MMAX_A,TXAB_VOR_MM_A,TXAB_VOR_MMAX_A,TXMIN_VOR_MM_A,TXMIN_VOR_MMAX_A,NBJFF10_MM_A,NBJFF10_MMAX_A,NBJFF10_MSOM_A,NBJFF16_MM_A,NBJFF16_MMAX_A,NBJFF16_MSOM_A,NBJFF28_MM_A,NBJFF28_MMAX_A,NBJFF28_MSOM_A,NBJFXI3S10_MM_A,NBJFXI3S10_MMAX_A,NBJFXI3S10_MSOM_A,NBJFXI3S16_MM_A,NBJFXI3S16_MMAX_A,NBJFXI3S16_MSOM_A,NBJFXI3S28_MM_A,NBJFXI3S28_MMAX_A,NBJFXI3S28_MSOM_A,NBJFXY8_MM_A,NBJFXY8_MMAX_A,NBJFXY8_MSOM_A,NBJFXY10_MM_A,NBJFXY10_MMAX_A,NBJFXY10_MSOM_A,NBJFXY15_MM_A,NBJFXY15_MMAX_A,NBJFXY15_MSOM_A,FFM_VOR_MM_A,FFM_VOR_MMAX_A,FXI3SAB_VOR_MM_A,FXI3SAB_VOR_MMAX_A,FXIAB_VOR_MM_A,FXIAB_VOR_MMAX_A,FXYAB_VOR_MM_A,FXYAB_VOR_MMAX_A,FFM_VOR_COM_MM_A_Y

Aperçu de y_train :


,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


Aperçu de X_test :


,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,CARACT5,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG8,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,CA1,CA2,CA3,KAPITAL1,KAPITAL2,KAPITAL3,KAPITAL4,KAPITAL5,KAPITAL6,KAPITAL7,KAPITAL8,KAPITAL9,KAPITAL10,KAPITAL11,KAPITAL12,KAPITAL13,KAPITAL14,KAPITAL15,KAPITAL16,KAPITAL17,KAPITAL18,KAPITAL19,KAPITAL20,KAPITAL21,KAPITAL22,KAPITAL23,KAPITAL24,KAPITAL25,KAPITAL26,KAPITAL27,KAPITAL28,KAPITAL29,KAPITAL30,KAPITAL31,KAPITAL32,KAPITAL33,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE2,SURFACE3,SURFACE4,SURFACE5,SURFACE6,SURFACE7,SURFACE8,SURFACE9,SURFACE10,SURFACE11,SURFACE12,SURFACE13,SURFACE14,SURFACE15,SURFACE16,SURFACE17,SURFACE18,SURFACE19,SURFACE20,SURFACE21,NBBAT1,NBBAT2,NBBAT3,NBBAT4,NBBAT5,NBBAT6,NBBAT7,NBBAT8,NBBAT9,NBBAT10,NBBAT11,NBBAT13,NBBAT14,TAILLE3,TAILLE4,NBSINCONJ,NBSINSTRT,COEFASS,RISK1,RISK2,RISK3,RISK4,RISK5,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT1,EQUIPEMENT2,EQUIPEMENT3,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,EQUIPEMENT7,DISTANCE_111,DISTANCE_112,DISTANCE_121,DISTANCE_122,DISTANCE_123,DISTANCE_124,DISTANCE_131,DISTANCE_132,DISTANCE_133,DISTANCE_141,DISTANCE_142,DISTANCE_211,DISTANCE_212,DISTANCE_213,DISTANCE_221,DISTANCE_222,DISTANCE_223,DISTANCE_231,DISTANCE_242,DISTANCE_243,DISTANCE_244,DISTANCE_311,DISTANCE_312,DISTANCE_313,DISTANCE_321,DISTANCE_322,DISTANCE_323,DISTANCE_324,DISTANCE_331,DISTANCE_332,DISTANCE_333,DISTANCE_334,DISTANCE_335,DISTANCE_411,DISTANCE_412,DISTANCE_421,DISTANCE_422,DISTANCE_423,DISTANCE_511,DISTANCE_512,DISTANCE_521,DISTANCE_522,DISTANCE_523,PROPORTION_11,PROPORTION_12,PROPORTION_13,PROPORTION_14,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,PROPORTION_32,PROPORTION_33,PROPORTION_41,PROPORTION_42,PROPORTION_51,PROPORTION_52,MEN,MEN_PAUV,MEN_1IND,MEN_5IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_INC,LOG_SOC,IND,IND_0_Y1,IND_Y1_Y2,IND_Y2_Y3,IND_Y3_Y4,IND_Y4_Y5,IND_Y5_Y6,IND_Y6_Y7,IND_Y7_Y8,IND_Y8_Y9,IND_Y9,IND_INC,IND_SNV,MEN_SURF,DISTANCE_1,DISTANCE_2,ALTITUDE_1,ALTITUDE_2,ALTITUDE_3,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,HAUTEUR,HAUTEUR_MAX,BDTOPO_BAT_MAX_HAUTEUR,ZONE_VENT,NB_CASERNES,NBJTX25_MM_A,NBJTX25_MMAX_A,NBJTX25_MSOM_A,NBJTX0_MM_A,NBJTX0_MMAX_A,NBJTX0_MSOM_A,NBJTXI27_MM_A,NBJTXI27_MMAX_A,NBJTXI27_MSOM_A,NBJTXS32_MM_A,NBJTXS32_MMAX_A,NBJTXS32_MSOM_A,NBJTXI20_MM_A,NBJTXI20_MMAX_A,NBJTXI20_MSOM_A,NBJTX30_MM_A,NBJTX30_MMAX_A,NBJTX30_MSOM_A,NBJTX35_MM_A,NBJTX35_MMAX_A,NBJTX35_MSOM_A,NBJTN10_MM_A,NBJTN10_MMAX_A,NBJTN10_MSOM_A,NBJTNI10_MM_A,NBJTNI10_MMAX_A,NBJTNI10_MSOM_A,NBJTN5_MM_A,NBJTN5_MMAX_A,NBJTN5_MSOM_A,NBJTNS25_MM_A,NBJTNS25_MMAX_A,NBJTNS25_MSOM_A,NBJTNI15_MM_A,NBJTNI15_MMAX_A,NBJTNI15_MSOM_A,NBJTNI20_MM_A,NBJTNI20_MMAX_A,NBJTNI20_MSOM_A,NBJTNS20_MM_A,NBJTNS20_MMAX_A,NBJTNS20_MSOM_A,NBJTMS24_MM_A,NBJTMS24_MMAX_A,NBJTMS24_MSOM_A,TAMPLIAB_VOR_MM_A,TAMPLIAB_VOR_MMAX_A,TAMPLIM_VOR_MM_A,TAMPLIM_VOR_MMAX_A,TM_VOR_MM_A,TM_VOR_MMAX_A,TMM_VOR_MM_A,TMM_VOR_MMAX_A,TMMAX_VOR_MM_A,TMMAX_VOR_MMAX_A,TMMIN_VOR_MM_A,TMMIN_VOR_MMAX_A,TN_VOR_MM_A,TN_VOR_MMAX_A,TNAB_VOR_MM_A,TNAB_VOR_MMAX_A,TNMAX_VOR_MM_A,TNMAX_VOR_MMAX_A,TX_VOR_MM_A,TX_VOR_MMAX_A,TXAB_VOR_MM_A,TXAB_VOR_MMAX_A,TXMIN_VOR_MM_A,TXMIN_VOR_MMAX_A,NBJFF10_MM_A,NBJFF10_MMAX_A,NBJFF10_MSOM_A,NBJFF16_MM_A,NBJFF16_MMAX_A,NBJFF16_MSOM_A,NBJFF28_MM_A,NBJFF28_MMAX_A,NBJFF28_MSOM_A,NBJFXI3S10_MM_A,NBJFXI3S10_MMAX_A,NBJFXI3S10_MSOM_A,NBJFXI3S16_MM_A,NBJFXI3S16_MMAX_A,NBJFXI3S16_MSOM_A,NBJFXI3S28_MM_A,NBJFXI3S28_MMAX_A,NBJFXI3S28_MSOM_A,NBJFXY8_MM_A,NBJFXY8_MMAX_A,NBJFXY8_MSOM_A,NBJFXY10_MM_A,NBJFXY10_MMAX_A,NBJFXY10_MSOM_A,NBJFXY15_MM_A,NBJFXY15_MMAX_A,NBJFXY15_MSOM_A,FFM_VOR_MM_A,FFM_VOR_MMAX_A,FXI3SAB_VOR_MM_A,FXI3SAB_VOR_MMAX_A,FXIAB_VOR_MM_A,FXIAB_VOR_MMAX_A,FXYAB_VOR_MM_A,FXYAB_VOR_MMAX_A,FFM_VOR_COM_MM_A_Y

# I - Feature engineering, column selection

Drop columns with more than 50% missing values

In [ ]:
# Calculation of the percentage of missing values per column
missing_percent = (X_train.isnull().sum() / X_train.shape[0]) * 100

# Select columns to drop (>50% missing values)
cols_to_drop = missing_percent[missing_percent > 50].index

# Column removal
X_train_drop = X_train.drop(columns=cols_to_drop, inplace=False)
X_test_drop = X_test.drop(columns=cols_to_drop, inplace=False)
print(cols_to_drop)

# Display of removed columns
# print(f"{len(cols_to_drop)} columns removed:", cols_to_drop.tolist())

Index(['CARACT2', 'CARACT3', 'TYPBAT1', 'DEROG12', 'DEROG13', 'DEROG14',
       'DEROG16', 'DISTANCE_111', 'DISTANCE_112', 'DISTANCE_121',
       ...
       'NBJRR30_MM_A', 'NBJRR30_MMAX_A', 'NBJRR30_MSOM_A', 'NBJRR100_MM_A',
       'NBJRR100_MMAX_A', 'NBJRR100_MSOM_A', 'RR_VOR_MM_A', 'RR_VOR_MMAX_A',
       'RRAB_VOR_MM_A', 'RRAB_VOR_MMAX_A'],
      dtype='object', length=183)


In [38]:
categorical_cols = X_train_drop.select_dtypes(include=['object', 'category']).columns
numeric_columns = X_train_drop.select_dtypes(include=['number']).columns
print(X_train_drop.shape[1])
print(len(categorical_cols))
print(len(numeric_columns))

191
98
93


Drop columns with more than 90% identical values

In [ ]:
import pandas as pd

# Liste des colonnes
all_cols = X_train_drop.columns
def cat_cols_to_keep(df, all_cols, threshold):
    cols_to_keep = []
    
    for col in all_cols:
        top_freq = df[col].value_counts(normalize=True).max() * 100  # % of the majority category
        if top_freq < threshold:  # If no category exceeds the threshold
            cols_to_keep.append(col)
    
    return cols_to_keep


cols_to_keep = cat_cols_to_keep(X_train_drop, all_cols, 90)

X_train_drop = X_train_drop.drop(columns=list(set(all_cols) - set(cols_to_keep)), inplace=False)
X_test_drop = X_test_drop.drop(columns=list(set(all_cols) - set(cols_to_keep)), inplace=False)

print(X_train_drop.shape)
print(X_test_drop.shape)

(383610, 97)
(95852, 97)


In [40]:
categorical_cols = X_train_drop.select_dtypes(include=['object', 'category']).columns
numeric_columns = X_train_drop.select_dtypes(include=['number']).columns
print(X_train_drop.shape[1])
print(len(categorical_cols))
print(len(numeric_columns))

97
53
44


Fill missing values by replacing NaNs with 0 in numerical columns and -999 in non-numerical columns.

In [ ]:
# Handling missing values in numerical columns
print("Processing missing values in numerical columns...")
numeric_columns = X_train_drop.select_dtypes(include=['number']).columns

# Processing missing values in numerical columns...
X_train_drop[numeric_columns] = X_train_drop[numeric_columns].fillna(0)
X_test_drop[numeric_columns] = X_test_drop[numeric_columns].fillna(0)

# Identify non-numerical columns
fill_cols = [item for item in X_train_drop.columns if item not in numeric_columns]

# Fill NaNs in non-numerical columns with a default value (-999)
X_train_drop[fill_cols] = X_train_drop[fill_cols].fillna(-999)
X_test_drop[fill_cols] = X_test_drop[fill_cols].fillna(-999)

print("Missing values processing completed.")

Traitement des valeurs manquantes dans les colonnes numériques...
Traitement des valeurs manquantes terminé.


Prepare the training data by removing unnecessary columns and encoding categorical variables using CountEncoder.

In [ ]:
# Data preparation for training
print("Data preparation for training...")

# Removal of unnecessary columns
X_train_drop = X_train_drop.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)
X_test_drop = X_test_drop.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)

# Encoding categorical variables with CountEncoder
encoder = CountEncoder(cols=fill_cols)
encoder.fit(X_train_drop)
X_train_enc = encoder.transform(X_train_drop)
X_test_enc = encoder.transform(X_test_drop)

print("Preparation completed.")

Préparation des données pour l'entraînement...
Préparation terminée.


In [43]:
X_test_enc 

,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,CARACT1,INDEM2,TYPBAT2,FRCH1,FRCH2,TAILLE1,TAILLE2,KAPITAL6,KAPITAL7,KAPITAL10,KAPITAL11,KAPITAL12,KAPITAL14,KAPITAL21,KAPITAL23,KAPITAL25,KAPITAL27,KAPITAL32,KAPITAL40,KAPITAL41,SURFACE1,SURFACE2,SURFACE3,SURFACE4,SURFACE5,SURFACE6,SURFACE7,SURFACE10,SURFACE11,SURFACE17,SURFACE18,NBBAT1,NBBAT4,NBBAT9,NBBAT13,NBBAT14,TAILLE3,NBSINCONJ,NBSINSTRT,COEFASS,RISK1,RISK2,RISK3,RISK4,RISK5,RISK6,RISK7,RISK8,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT2,EQUIPEMENT4,EQUIPEMENT6,EQUIPEMENT7,PROPORTION_11,PROPORTION_21,PROPORTION_22,PROPORTION_23,PROPORTION_24,PROPORTION_31,MEN_PAUV,MEN_1IND,MEN_PROP,MEN_FMP,MEN_COLL,MEN_MAIS,LOG_AVA1,LOG_A1_A2,LOG_A2_A3,LOG_APA3,LOG_SOC,IND_Y3_Y4,IND_Y6_Y7,IND_Y8_Y9,IND_Y9,IND_SNV,MEN_SURF,ALTITUDE_2,ALTITUDE_3,ALTITUDE_4,ALTITUDE_5,ZONE_VENT,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,FXI3SAB_VOR_COM_MMAX_A_Y,ESPINSEE,AN_EXERC,ZONE
0,296563,244441,2,4,93158,3404,0,102148,26573,79214,95799,0,0,0,0.0,125000,0,0.0,0.0,1000,25000,87500,220059,319270,750,750,0.0,97717,50,97717,0.0,0.0,0.0,0,0.0,5,4,0,4,1,0,0.00,2.00,340077,0.0,0.0,0.0,0.0,0.0,26573,0.0,26573,87340,85903,26573,156466,87969,1,5,6,319870,33049,312169,59057,163562,17500,115306,160610,124689,112528,236355,227485,73293,250558,132692,114582,319281,60165,173488,273630,320197,148819,128509,90698,90415,105864,86817,2.0,125845,130869,281064,281096,151542,26573,3
1,296563,244441,2,6,93158,67948,0,102148,26573,126652,86531,0,0,0,0.0,125000,0,0.0,0.0,4000,0,62500,220059,319270,175,175,0.0,75374,0,75374,0.0,50.0,0.0,0,0.0,4,3,3,3,0,1,0.00,0.00,340077,0.0,0.0,0.0,0.0,0.0,26573,0.0,26573,87340,85903,26573,156466,87969,1,5,6,319870,134467,312169,28098,163562,9976,115306,147462,124689,252432,236355,227485,54065,250558,132692,114582,319281,60165,173488,273630,320197,148819,128509,33470,33554,38514,32469,1.0,125845,130869,281064,281096,151542,26573,3
2,296563,38967,2,2,274335,67948,0,102148,26573,126652,66538,0,0,0,0.0,125000,0,0.0,0.0,11000,0,0,220059,319270,100,100,0.0,75374,50,75374,0.0,50.0,0.0,0,0.0,2,1,1,1,2,0,0.00,0.00,340077,0.0,0.0,0.0,0.0,0.0,26573,0.0,26573,140744,140728,26573,156466,87969,1,1,2,319870,33039,312169,21040,163562,149794,115306,160610,124689,252432,236355,227485,73293,250558,132692,94401,319281,60165,173488,273630,320197,148819,128509,235954,236124,105864,240256,2.0,125845,130869,80715,80684,151542,26573,3
3,14359,14359,2,2,274335,67948,0,102148,26573,50552,55253,0,0,0,0.0,0,0,0.0,0.0,3000,0,0,220059,319270,900,900,0.0,59031,275,59031,0.0,50.0,0.0,0,0.0,7,6,5,6,1,0,0.00,0.00,340077,0.0,0.0,0.0,0.0,0.0,26573,0.0,26573,140744,140728,26573,156466,87969,1,1,2,319870,31796,312169,37268,163562,36908,115306,160610,156440,252432,236355,227485,73293,250558,164016,114582,319281,304855,173488,273630,320197,53263,128509,90698,90415,105864,86817,2.0,125845,130869,80715,80684,151542,26573,3
4,14359,14359,2,3,93158,215687,0,102148,26573,42991,55253,0,0,0,0.0,0,0,0.0,0.0,0,0,0,220059,319270,1900,1900,0.0,26769,0,26769,0.0,100.0,0.0,0,0.0,7,6,2,6,0,0,0.00,0.00,340077,0.0,0.0,0.0,0.0,0.0,26573,0.0,26573,87340,85903,26573,156466,87969,1,3,4,319870,31796,312169,37268,37000,149794,115306,160610,57230,112528,236355,227485,66230,250558,164016,48159,36778,60165,191428,273630,320197,148819,151993,235954,236124,105864,240256,2.0,125845,130869,80715,80684,151542,26573,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95847,296563,244441,1,11,274335,215687,1,73268,123820,5113,2918,1,0,450000,450000.0,450000,0,6000.0,0.0,1000,50000,250000,163551,319270,8500,8500,8500.0,8227,0,8228,100.0,150.0,8500.0,50,0.0,12,11,0,11,0,0,0.62,0.93,340077,3.0,27.0,9.0,3.0,4.0,197833,85.0,115981,79901,1569

# II - Models

Claim frequency model

In [ ]:
# Train the models
print("Training the models")

# Model to predict 'FREQ'
rf_freq = RandomForestRegressor(random_state=42, n_estimators=9)
rf_freq.fit(X_train_enc, y_train['FREQ'])
# print("Model for 'FREQ' trained successfully.")

# 2 min 16.7 sec

Entraînement des modèles
Modèle pour 'FREQ' entraîné avec succès.


In [ ]:
# Check the depth of each tree
# for i, tree in enumerate(rf_freq.estimators_):
# tree_depth = tree.get_depth() # No need to sub-index here
# print(f"Tree {i} has a depth of {tree_depth}")
# Tree 0 has a depth of 37
# Tree 1 has a depth of 49
# Tree 2 has a depth of 34
# Tree 3 has a depth of 42
# Tree 4 has a depth of 42
# Tree 5 has a depth of 37
# Tree 6 has a depth of 44
# Tree 7 has a depth of 40
# Tree 8 has a depth of 40

In [ ]:
# Predict 'FREQ' on the training set
y_train_pred_freq = rf_freq.predict(X_train_enc)

Average claim cost model

In [ ]:
import lightgbm as lgb
from sklearn import model_selection

# Data splitting
target_cm = y_train['CM'].loc[X_train.index].copy()
X_train_cm, X_test_cm, y_train_cm, y_test_cm = model_selection.train_test_split(X_train_enc, target_cm, train_size=0.8, random_state=42)
X_train_cm, X_valid_cm, y_train_cm, y_valid_cm = model_selection.train_test_split(X_train_cm, y_train_cm, train_size=0.8, random_state=42)

# LightGBM parameters
params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'regression',  
    'metric': 'l2', 
    'learning_rate': 0.006,
    'max_depth': 3,
    'verbose': -1
    
}

d_train = lgb.Dataset(X_train_cm, label=y_train_cm)
d_valid = lgb.Dataset(X_valid_cm, label=y_valid_cm, reference=d_train)

# Training
gbm_cm = lgb.train(
    params_lgb,
    d_train,
    num_boost_round=10000,
    valid_sets=[d_train, d_valid],
    valid_names=['train', 'eval'],
)

# Prediction
y_train_pred_cm = gbm_cm.predict(X_train_enc, num_iteration=gbm_cm.best_iteration)
y_train_pred_cm

# III - Model evaluation and prediction

In [ ]:
# Calculate the combined prediction for 'CHARGE'
y_train_pred = y_train_pred_freq * y_train_pred_cm * y_train['ANNEE_ASSURANCE']

In [ ]:
# Calculate RMSE on the training set
rmse = np.sqrt(mean_squared_error(y_train['CHARGE'], y_train_pred))
print(f"RMSE on the training set : {rmse:.4f}")

RMSE sur l'ensemble d'entraînement : 6755.8962


In [ ]:
# Predictions on the test set
print("Predictions on the test set...")

# freq with Random Forest
y_pred_freq = rf_freq.predict(X_test_enc)

# cm with LightGBM
y_pred_cm = gbm_cm.predict(X_test_enc, num_iteration=gbm_cm.best_iteration)

Prédictions sur l'ensemble de test...


In [ ]:
# Combine the predictions
y_pred = pd.concat([
    X_test[['ID', 'ANNEE_ASSURANCE']].reset_index(drop=True),
    pd.DataFrame(y_pred_freq, columns=['FREQ']),
    pd.DataFrame(y_pred_cm, columns=['CM'])
], axis=1)

In [ ]:
# Calculate the 'CHARGE' column by multiplying 'FREQ', 'CM', and 'ANNEE_ASSURANCE'
y_pred['CHARGE'] = y_pred['CM'] * y_pred['FREQ'] * y_pred['ANNEE_ASSURANCE']

print("Predictions completed.")

Prédictions terminées.


# IV - Submission

In [ ]:
# Export predictions to a CSV file
print("Exporting results...")
y_pred.to_csv('submission.csv', index=False)
print("Submission file created: 'submission.csv'")

Exportation des résultats...
Fichier de soumission créé : 'submission.csv'
